This notebook implementation incorporates elements from [this](https://keras.io/examples/vision/mnist_convnet/) Keras example.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from fkan.tensorflow import FractionalJacobiNeuralBlock as fJNB
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

2024-06-11 23:36:55.965039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


# assume x_train is your original dataset
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [3]:
batch_size = 512
epochs = 30
q = 2  # See paper for the definition and role of q
trial = 1

If using a predefined Keras activation function, replace each `fJNB(q)` with: `layers.Activation(activation)`

In [4]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3)),
        fJNB(q),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3)),
        fJNB(q),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_valid, y_valid),
    verbose=1,
)
score = model.evaluate(x_test, y_test, verbose=1)
print(history.history, "\n", score, file=open("data/history-JNB%s-%d.txt" % (q, trial), "w"))

2024-06-11 23:36:57.458447: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


313/313 [==============================] - 3s 9ms/step - loss: 0.9438 - accuracy: 0.8084
